## 0. Import lib, dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
% matplotlib inline
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.preprocessing import RobustScaler

In [2]:
data_train = pd.read_csv('/content/drive/MyDrive/python/Rolling in the deep/train.csv')

## 1. Data Preprocessing

In [3]:
data_train = data_train.fillna('unknown')

X= data_train.drop(columns=['index', 'FLAG_MOBIL', 'child_num', 'credit'])
y= data_train['credit']

X_encoded = pd.get_dummies(X, columns=['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type'])

In [4]:
num_features = [(0, 'income_total'), (1, 'DAYS_BIRTH'), (2, 'DAYS_EMPLOYED'), (6, 'family_size'), (7, 'begin_month')]
already = [0, 1, 2, 6, 7]
features = list(range(54))

for i in already:
  features.remove(i)

scaler = RobustScaler()
X_dummy = X_encoded.iloc[:, features].reset_index(drop=True)
X_robust = pd.DataFrame(scaler.fit_transform(X_encoded.iloc[:, [t[0] for t in num_features]]), columns=[t[1] for t in num_features])
X_final = pd.concat([X_robust,X_dummy ], axis=1)

In [5]:
X_final

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,family_size,begin_month,work_phone,phone,email,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,0.434783,0.235934,-1.154406,0.0,0.666667,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.869565,0.596564,-0.000364,1.0,0.703704,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2.826087,-0.506800,-1.054261,0.0,0.074074,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0.434783,0.065712,-0.201384,0.0,-0.481481,0,1,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0.000000,0.073014,-0.206118,0.0,-0.074074,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0.652174,0.496492,-0.162054,2.0,0.814815,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26453,0.217391,0.036650,-0.340859,0.0,-0.851852,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26454,1.304348,0.782391,-0.173343,0.0,-0.037037,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26455,0.130435,0.773372,0.521486,-1.0,-1.296296,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [7]:
list = [X_train, X_valid, X_test, y_train, y_valid, y_test]
for i in list:
  print(i.shape)

(15873, 54)
(5292, 54)
(5292, 54)
(15873,)
(5292,)
(5292,)


## 2. Classification

In [8]:
from sklearn.model_selection import GridSearchCV as cv
from xgboost import plot_importance
from xgboost import XGBClassifier

In [9]:
xgb_params = {'eta':(0.01, 0.1, 1), 'max_depth':range(10,50,10)}

In [10]:
clf_xgb = GridSearchCV(XGBClassifier(), param_grid=xgb_params)

In [11]:
clf_xgb.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'eta': (0.01, 0.1, 1), 'max_depth': range(10, 50, 10)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scor

In [12]:
y_pred_xgb = clf_xgb.predict_proba(X_valid)
log_loss(y_true=y_valid, y_pred=y_pred_xgb)

0.758077860912287

In [16]:
y_pred_xgb_label = clf_xgb.predict(X_valid)

from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_true = y_valid, y_pred = y_pred_xgb_label)

array([[[4606,   42],
        [ 590,   54]],

       [[3898,  150],
        [ 876,  368]],

       [[ 534, 1354],
        [  80, 3324]]])

In [21]:
 from sklearn.metrics import classification_report
 print(classification_report(y_true = y_valid,y_pred = y_pred_xgb_label))

              precision    recall  f1-score   support

         0.0       0.56      0.08      0.15       644
         1.0       0.71      0.30      0.42      1244
         2.0       0.71      0.98      0.82      3404

    accuracy                           0.71      5292
   macro avg       0.66      0.45      0.46      5292
weighted avg       0.69      0.71      0.65      5292

